In [1]:
from lxml import etree
import pandas as pd
# import dask.dataframe as dd

In [ ]:
file_path = "C:/Users/Настя/YandexDisk-n4skolesnikova/HSE 4th year/Graduation Thesis/gibdd_parser/storage/ДТП_"
regions = [
    1,
    3,
    4,
    5,
    7,
    8,
    10,
    11,
    12,
    14,
    15,
    17,
    18,
    19,
    20,
    21,
    22,
    23,
    24,
    25,
    26,
    27,
    28,
    29,
    30,
    32,
    33,
    34,
    35,
    36,
    37,
    38,
    40,
    41,
    42,
    43,
    44,
    45,
    46,
    47,
    49,
    50,
    52,
    53,
    54,
    55,
    56,
    57,
    58,
    60,
    61,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80,
    81,
    82,
    83,
    84,
    85,
    86,
    87,
    88,
    89,
    90,
    91,
    92,
    93,
    94,
    95,
    96,
    97,
    98,
    99,
    71100,
    71140,
    10011,
]
years = range(2015, 2025)
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

russians = ['VIS', 'Bronto', 'Lada', 'GAZ', 'ГАЗ', 'Газ', 'Урал', 'ПАЗ', 'КАВЗ', 'ЛиАЗ', 'ГОЛАЗ', 'КАМАЗ', 'НефАЗ', 'VDL', 'Бравис',\
            'УАЗ', 'Ssang', 'ЗИЛ', 'Баз', 'Грейт', 'Opel', 'Volgabus', 'BAW', 'Tar', 'Lifan', 'Geely', 'Блеск', 'Chery', 'Jianghuai', 'Хавтай']

In [ ]:
data = []
dropped_rta = 0

for region in regions:
    for year in years:
        for month in months:
            try:
                file_name = file_path + str(region) + '_' + str(year) + '_' + month + '.xml'
                tree = etree.parse(file_name)

                root = tree.getroot()

                # loop over accidents for the n-th month of the m-th year in the k-th region
                for tab in root.iter('tab'):
                    kuch = tab.find('KUCH')
                    if kuch is not None and kuch.text and kuch.text.isdigit():
                        if int(kuch.text) >= 10:
                            dropped_rta += 1
                            continue
                    kts = tab.find('KTS')       # num of vehicles in accident
                    # removing outliers
                    if kts is not None and kts.text and kts.text.isdigit():
                        if int(kts.text) >= 5:
                            dropped_rta += 1
                            continue

                    record = {}
                    for element in tab.iter():
                        if element.tag not in {'tab', 'infoDtp', 'ts_info', 'ts_uch', 'uchInfo'}:
                            record[element.tag] = element.text
                        elif element.tag == 'infoDtp':
                            for info in element.iter():
                                if info.tag not in {'infoDtp', 'ts_info', 'ts_uch', 'uchInfo'}:
                                    record[info.tag] = info.text


                    # creating binary features
                    record['any TV technical failures'] = 0
                    record['any non-private vehicles'] = 0
                    record['any russian vehicles'] = 0

                    record['any white vehicles'] = 0
                    record['any black vehicles'] = 0
                    record['any colored vehicles'] = 0

                    record['any drunk drivers'] = 0
                    record['any female drivers'] = 0
                    record['any escaped participants'] = 0

                    record['any concomitant traffic violations'] = 0
                    record['any injuries without safety belt'] = 0


                    # creating non-binary features
                    record['num drunk participants'] = 0
                    record['num child restraints'] = 0
                    record['num cyclists'] = 0
                    record['num pedestrians'] = 0

                    record['min vehicles age'] = 0
                    record['max vehicles age'] = 0
                    record['avg vehicles age'] = 0
                    lst_age = []        # auxiliary element

                    record['num A-class vehicles'] = 0
                    record['num B-class vehicles'] = 0
                    record['num C-class vehicles'] = 0
                    record['num D-class vehicles'] = 0
                    record['num E-class vehicles'] = 0
                    record['num S-class vehicles'] = 0
                    record['num non-car vehicles'] = 0

                    record['num front-wheel-drive vehicles'] = 0
                    record['num rear-wheel-drive vehicles'] = 0
                    record['num four-wheel-drive vehicles'] = 0

                    record['num guilty objects'] = 0
                    record['share guilty vehicles'] = 0
                    num_of_guilty_drivers = 0       # auxiliary element
                    record['fatal traffic violation(s)'] = None
                    record['num fatal traffic violations'] = 0
                    record['bodily harm'] = None

                    record['guilty drivers\' avg experience'] = None
                    guilty_drivers_experience = 0       # auxiliary element
                    record['drivers\' avg experience'] = None
                    drivers_experience = 0      # auxiliary element

                    # auxiliary elements
                    violations = set()
                    injuries = set()

                    # loop over vehicles in the accident
                    for ts_info in tab.iter('ts_info'):

                        failures = ts_info.find('t_n')
                        if failures is not None and failures.text and failures.text != 'Технические неисправности отсутствуют':
                            record['any TV technical failures'] = 1

                        property = ts_info.find('f_sob')
                        if property is not None and property.text and property.text != 'Частная собственность':
                            record['any non-private vehicles'] = 1

                        year_of_release = ts_info.find('g_v')
                        if year_of_release is not None and year_of_release.text and year_of_release.text.isdigit():
                            lst_age.append(int(year_of_release.text))

                        color = ts_info.find('color')
                        if color is not None and color.text:
                            if color.text == 'Белый':
                                record['any white vehicles'] = 1
                            elif color.text == 'Черный':
                                record['any black vehicles'] = 1
                            else:
                                record['any colored vehicles'] = 1

                        wheel = ts_info.find('r_rul')
                        if wheel is not None:
                            if wheel.text == 'С передним приводом':
                                record['num front-wheel-drive vehicles'] += 1
                            elif wheel.text == 'С задним приводом':
                                record['num rear-wheel-drive vehicles'] += 1
                            elif wheel.text == 'Полноприводные':
                                record['num four-wheel-drive vehicles'] += 1

                        vehicle_class = ts_info.find('t_ts')
                        if vehicle_class is not None and vehicle_class.text:
                            if vehicle_class.text == 'А-класс (особо малый) до 3,5 м':
                                record['num A-class vehicles'] += 1
                            elif vehicle_class.text == 'В-класс (малый) до 3,9 м':
                                record['num B-class vehicles'] += 1
                            elif vehicle_class.text == 'С-класс (малый средний, компактный) до 4,3 м':
                                record['num C-class vehicles'] += 1
                            elif vehicle_class.text == 'D-класс (средний) до 4,6 м':
                                record['num D-class vehicles'] += 1
                            elif vehicle_class.text == 'Е-класс (высший средний, бизнес-класс) до 4,9 м':
                                record['num E-class vehicles'] += 1
                            elif vehicle_class.text == 'S-класс (высший, представительский класс) более 4,9 м':
                                record['num S-class vehicles'] += 1

                        model = ts_info.find('m_ts')
                        if model is not None and model.text:
                            for rus in russians:
                                if rus in model:
                                    record['any russian vehicles'] = 1
                                    continue


                        is_vehicle_guilty = 0

                        # loop over passengers
                        for ts_uch in ts_info.iter('ts_uch'):

                            drunk = ts_uch.find('ALCO')                                
                            participant = ts_uch.find('k_UCH')
                            gender = ts_uch.find('POL')

                            if participant is not None:
                                if participant.text == 'Водитель' and drunk is not None and drunk.text:
                                    record['any drunk drivers'] = 1
                                    record['num drunk participants'] += 1
                                elif participant.text == 'Пассажир' and drunk is not None and drunk.text:
                                    record['num drunk participants'] += 1
                                elif participant.text == 'Велосипедист':
                                    record['num cyclists'] += 1
                                
                                if gender is not None and gender.text == 'Женский' and participant.text == 'Водитель':
                                    record['any female drivers'] = 1

                            leaving = ts_uch.find('s_SM')
                            if leaving is not None and leaving.text and leaving.text != 'Нет (не скрывался)':
                                record['any escaped participants'] = 1

                            concomitant_violations = ts_uch.find('SOP_NPDD')
                            if concomitant_violations is not None and concomitant_violations.text and concomitant_violations.text != 'Нет нарушений':
                                record['any concomitant traffic violations'] = 1

                            child = ts_uch.find('s_SEAT_GROUP')
                            if child is not None and child.text:
                                record['num child restraints'] += 1

                            safety_belt = ts_uch.find('SAFETY_BELT')
                            injury = ts_uch.find('s_T')
                            if injury is not None and injury.text and injury.text != 'Не пострадал':
                                injuries.add(injury.text)
                                if safety_belt is not None and safety_belt.text == 'Нет':
                                    record['any injuries without safety belt'] = 1

                            is_obj_guily = 0

                            for guilty in ts_uch.iter('NPDD'):
                                if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                                    is_vehicle_guilty = 1
                                    violations.add(guilty.text)

                            experience = ts_uch.find('v_ST')
                            if experience is not None and experience.text and experience.text.isdigit():
                                drivers_experience += int(experience.text)
                                if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                                    guilty_drivers_experience += int(experience.text)

                        record['num guilty objects'] += is_vehicle_guilty
                        num_of_guilty_drivers += is_vehicle_guilty

                    if num_of_guilty_drivers != 0:  
                        record['guilty drivers\' avg experience'] = guilty_drivers_experience / num_of_guilty_drivers
                    record['drivers\' avg experience'] = drivers_experience / int(kts.text)

                    if lst_age:
                        record['avg vehicles age'] = sum(lst_age) / len(lst_age)
                        record['min vehicles age'] = min(lst_age)
                        record['max vehicles age'] = max(lst_age)
                    else:
                        record['avg vehicles age'] = None
                        record['min vehicles age'] = None
                        record['max vehicles age'] = None

                    pedestrian = tab.find('uchInfo')
                    if pedestrian is not None:
                        record['num pedestrians'] += 1


                    guilty_objects = 0

                    # loop over pedestrians (outside vehicles)
                    for uchInfo in tab.iter('uchInfo'):

                        leaving = uchInfo.find('s_SM')
                        if leaving is not None and leaving.text and leaving.text != 'Нет (не скрывался)':
                            record['any escaped participants'] = 1

                        concomitant_violations = uchInfo.find('SOP_NPDD')
                        if concomitant_violations is not None and concomitant_violations.text and concomitant_violations.text != 'Нет нарушений':
                            record['any concomitant traffic violations'] = 1

                        is_obj_guily = 0

                        for guilty in uchInfo.iter('NPDD'):
                            if guilty is not None and guilty.text and guilty.text != 'Нет нарушений':
                                is_obj_guily = 1
                                violations.add(guilty.text)
                        guilty_objects += is_obj_guily

                        injury = uchInfo.find('s_T')
                        if injury is not None and injury.text and injury.text != 'Не пострадал':
                            injuries.add(injury.text)

                    record['num guilty objects'] += guilty_objects
                    record['share guilty vehicles'] = record['num guilty objects'] / int(kts.text)
                    record['fatal traffic violation(s)'] = violations
                    record['number fatal traffic violations'] = len(violations)

                    if len(injuries) != 0:
                        record['bodily harm'] = injuries
                    else:
                        record['bodily harm'] = {'Не пострадал'}

                    data.append(record)

            except OSError:
                continue

print(dropped_rta)
df_correct = pd.DataFrame(data)
df_correct.to_csv('DTP_DATA_2025.csv')
df_correct

11193


,DTPV,date,district,EMTP_NUMBER,CHOM,COORD_L,COORD_W,dor,dor_k,dor_z,...,num rear-wheel-drive vehicles,num four-wheel-drive vehicles,num guilty objects,share guilty vehicles,fatal traffic violation(s),num fatal traffic violations,bodily harm,guilty drivers' avg experience,drivers' avg experience,number fatal traffic violations
0,Столкновение,31.01.2015,ЖЕЛЕЗНОДОРОЖНЫЙ,None,Движение частично перекрыто,83.740833,53.333056,None,None,Не указано,...,0,0,1,0.5,{Другие нарушения ПДД водителем},0,"{Раненый, находящийся (находившийся) на амбул...",15.0,13.0,1
1,Наезд на пешехода,31.01.2015,АЛЕЙСКИЙ,None,Режим движения не изменялся,82.462778,52.245833,А-322 Барнаул - Рубцовск - граница с Республик...,3,Федеральная (дорога федерального значения),...,0,0,1,1.0,{Нахождение на проезжей части без цели её пере...,0,"{Раненый, находящийся (находившийся) на стацио...",NaN,5.0,1
2,Наезд на пешехода,31.01.2015,ИНДУСТРИАЛЬНЫЙ,None,Движение частично перекрыто,83.501944,53.274167,None,None,Не указано,...,0,0,2,2.0,{Переход через проезжую часть вне пешеходного ...,0,"{Раненый, находящийся (находившийся) на амбул...",NaN,17.0,1
3,Наезд на пешехода,31.01.2015,ЦЕНТРАЛЬНЫЙ,None,Режим движения не изменялся,83.251667,53.285278,None,None,Не указано,...,0,0,1,1.0,{Переход через проезжую часть вне пешеходного ...,0,"{Раненый, находящийся (находившийся) на амбул...",NaN,27.0,1
4,Наезд на пешехода,31.01.2015,ИНДУСТРИАЛЬНЫЙ,None,Движение частично перекрыто,83.708889,53.318333,None,None,Не указано,...,0,0,1,1.0,{Переход через проезжую часть в неустановленно...,0,"{Раненый, находящийся (находившийся) на стацио...",NaN,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475669,Съезд с дороги,04.07.2024,Заполярный район,510000253,Режим движения не изменялся,53.607541,67.836463,Подъездная автомобильная дорога к п. Красное,1,Региональная или межмуниципальная (дорога реги...,...,0,0,1,1.0,{Несоответствие скорости конкретным условиям д...,0,"{Раненый, находящийся (находившийся) на стацио...",24.0,24.0,1
1475670,Опрокидывание,04.07.2024,Заполярный район,510000254,Режим движения не изменялся,53.594570,67.768850,None,None,"Местного значения (дорога местного значения, в...",...,0,0,1,1.0,{Несоответствие скорости конкретным условиям д...,0,"{Раненый, находящийся (находившийся) на стацио...",25.0,25.0,1
1475671,Наезд на препятствие,25.08.2024,Заполярный район,510000306,Режим движения не изменялся,53.612724,67.837285,Подъездная автомобильная дорога к п. Красное,4,Региональная или межмуниципальная (дорога реги...,...,0,0,1,1.0,{Несоответствие скорости конкретным условиям д...,0,{Получил телесные повреждения с показанием к л...,95.0,95.0,1
1475672,Наезд на стоящее ТС,24.09.2024,Заполярный район,510000328,Режим движения не изменялся,55.956877,67.303057,"г. Нарьян-Мар - г. Усинск, участок п. Харьягин...",8,Региональная или межмуниципальная (дорога реги...,...,1,1,1,0.5,{Нарушение правил расположения ТС на проезжей ...,0,"{Раненый, находящийся (находившийся) на стацио...",5.0,2.5,1
